In [2]:


!pip install plotly



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\binde\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
!pip install nbformat --upgrade


In [9]:
import pandas as pd
from datetime import datetime as dt, timedelta
import plotly.express as px
import plotly.graph_objects as go
import plotly.colors

In [16]:
data = pd.read_csv(r"c:\Users\binde\Documents\Python Projects\online_retail.csv")
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [17]:
data.tail()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [18]:
data.dropna(subset=['CustomerID'], inplace=True)

In [19]:
data['InvoiceDate']=pd.to_datetime(data['InvoiceDate'])
data['TotalAmount']=data['Quantity']* data['UnitPrice']


In [20]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalAmount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [21]:
reference_date=pd.Timestamp(dt.now().date())

In [22]:
reference_date=data['InvoiceDate'].max()+timedelta(days=1)

In [23]:
reference_date

Timestamp('2011-12-10 12:50:00')

In [24]:
rfm = data.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (reference_date-x.max()).days,
    'InvoiceNo' : 'count',
    'TotalAmount': 'sum'
})


In [25]:
rfm.rename(columns={'InvoiceDate':'Recency', 'InvoiceNo': 'Frequency', 'TotalAmount':'Value'}, inplace=True)
rfm.head()

,Recency,Frequency,Value
CustomerID,,,
12346.0,326,2,0.00
12347.0,2,182,4310.00
12348.0,75,31,1797.24
12349.0,19,73,1757.55
12350.0,310,17,334.40


In [28]:
# defining quantiles
quantiles = rfm.quantile(q=[0.25,0.5,0.75])

#Assigning RFM Scores
def Rscore(x, p, d):
    if p=='Recency':
        if x <=d[p][0.25]:
            return 4
        elif x<=d[p][0.50]:
            return 3
        elif x<=d[p][0.75]:
            return 2
        else:
            return 1
    else:
        if x <=d[p][0.25]:
            return 1
        elif x<=d[p][0.50]:
            return 2
        elif x<=d[p][0.75]:
            return 3
        else:
            return 4
        
rfm['R']=rfm['Recency'].apply(Rscore, args=('Recency', quantiles,))
rfm['F']=rfm['Frequency'].apply(Rscore, args=('Frequency', quantiles,))
rfm['M']=rfm['Value'].apply(Rscore, args=('Value', quantiles,))

rfm.head()

,Recency,Frequency,Value,R,F,M
CustomerID,,,,,,
12346.0,326,2,0.00,1,1,1
12347.0,2,182,4310.00,4,4,4
12348.0,75,31,1797.24,2,2,4
12349.0,19,73,1757.55,3,3,4
12350.0,310,17,334.40,1,1,2


In [29]:
#consolidated RFM scores
rfm['RFM_Segment']=rfm['R'].astype(str)+rfm['F'].astype(str)+rfm['M'].astype(str)
#RFM score=total score
rfm['RFM_Score']=rfm[['R', 'F', 'M']].sum(axis=1)

In [30]:
rfm.head()

,Recency,Frequency,Value,R,F,M,RFM_Segment,RFM_Score
CustomerID,,,,,,,,
12346.0,326,2,0.00,1,1,1,111,3
12347.0,2,182,4310.00,4,4,4,444,12
12348.0,75,31,1797.24,2,2,4,224,8
12349.0,19,73,1757.55,3,3,4,334,10
12350.0,310,17,334.40,1,1,2,112,4


In [31]:
# Segmenting the RFM scores
segment_labels=['Low-Value', 'Mid-Value', 'High-Value']

def assign_segment(score):
    if score < 5:
        return 'Low_Value'
    elif score < 9:
        return 'Mid_Value'
    else:
        return 'High-Value'

rfm['RFM_Segment_Label']= rfm['RFM_Score'].apply(assign_segment)

rfm.head()

,Recency,Frequency,Value,R,F,M,RFM_Segment,RFM_Score,RFM_Segment_Label
CustomerID,,,,,,,,,
12346.0,326,2,0.00,1,1,1,111,3,Low_Value
12347.0,2,182,4310.00,4,4,4,444,12,High-Value
12348.0,75,31,1797.24,2,2,4,224,8,Mid_Value
12349.0,19,73,1757.55,3,3,4,334,10,High-Value
12350.0,310,17,334.40,1,1,2,112,4,Low_Value


In [42]:
segment_counts=rfm['RFM_Segment_Label'].value_counts().reset_index()
segment_counts.columns =['RFM_Segment', 'Count']
segment_counts=segment_counts.sort_values('RFM_Segment')

print(segment_counts)

  RFM_Segment  Count
1  High-Value   1690
2   Low_Value    783
0   Mid_Value   1899


In [45]:
#bar chart using PLotly
fig = px.bar(segment_counts,
             x='RFM_Segment',
             y='Count',
             title='Customer Distribution by RFM Segment',
             labels= {'RFM_Segment': 'RFM Segment', 'Count': 'Number of Customers'},
             color='RFM_Segment',
             color_discrete_sequence=px.colors.qualitative.Pastel)
fig.show()

In [51]:
rfm['RFM_Customer_Segments']=''
rfm.loc[rfm['RFM_Score']>=9, 'RFM_Customer_Segments']='SuperLoyal'
rfm.loc[(rfm['RFM_Score']>=6) & (rfm['RFM_Score']<9),'RFM_Customer_Segments']='Potential Loyal'
rfm.loc[(rfm['RFM_Score']>=5) & (rfm['RFM_Score']<6),'RFM_Customer_Segments']='At Risk Customers'
rfm.loc[(rfm['RFM_Score']>=4) & (rfm['RFM_Score']<5),'RFM_Customer_Segments']="Can't lose"
rfm.loc[(rfm['RFM_Score']>=3) & (rfm['RFM_Score']<4),'RFM_Customer_Segments']="Lost"
segment_counts=rfm['RFM_Customer_Segments'].value_counts().sort_index()

In [54]:
segment_product_counts=rfm.groupby(['RFM_Segment_Label', 'RFM_Customer_Segments']).size().reset_index(name='Count')

segment_product_counts=segment_product_counts.sort_values('Count', ascending=False)

In [55]:
fig_treemap = px.treemap(segment_product_counts,
             path=['RFM_Segment_Label','RFM_Customer_Segments'],
             values='Count',
             color='RFM_Segment_Label',
             color_discrete_sequence=px.colors.qualitative.Pastel,
             title='RFM Customer Segments by Value')
fig_treemap.show()

In [57]:
#For Recency, Frequency anbd Monetary Value for the Super Loyal Customers
SuperLoyal_segment=rfm[rfm['RFM_Customer_Segments'] =='SuperLoyal']

In [58]:
fig = go.Figure()
fig.add_trace(go.Box(y=SuperLoyal_segment['Recency'], name='Recency'))
fig.add_trace(go.Box(y=SuperLoyal_segment['Frequency'], name='Frequency'))
fig.add_trace(go.Box(y=SuperLoyal_segment['Value'], name='Value'))


In [59]:
#Correlation matrix
correlation_matrix=SuperLoyal_segment[['R',"F","M"]].corr()

In [66]:
#HeatMap to Visualzie the correlation

fig_heatmap=go.Figure(data=go.Heatmap(
    z=correlation_matrix.values,
    x=correlation_matrix.columns,
    y=correlation_matrix.columns,
    colorscale='RdBu',
    colorbar=dict(title='Correlation')))
fig_heatmap.update_layout(title='Correlation Matrix of RFM Values within Super Loyal Segment')
fig_heatmap.show()

In [67]:
pastel_colors=plotly.colors.qualitative.Pastel

fig=go.Figure(data=[go.Bar(x=segment_counts.index, y=segment_counts.values, marker=dict(color=pastel_colors))])

vip_color='rgb(158,202,225)'

fig.update_traces(marker_color=[vip_color if segment == 'SuperLoyal' else pastel_colors[i]
                                for i, segment in enumerate(segment_counts.index)],
                  marker_line_color='rgb(8, 48, 107)',
                  marker_line_width=1.5, opacity=0.6)

#Update Layout
fig.update_layout(title='Comparison of RFM Segments',
                  xaxis_title='RFM Segments',
                  yaxis_title='Number of Customers',
                  showlegend=False)

#Display the Figure
fig.show()

In [69]:
segment_scores=rfm.groupby('RFM_Customer_Segments')[['R', 'F', 'M']].mean().reset_index()
fig=go.Figure()

#add Bars for Recency Score
fig.add_trace(go.Bar(
    x=segment_scores['RFM_Customer_Segments'],
    y=segment_scores['R'],
    name='Recency Score',
    marker_color='rgb(158,202,255)'
))

#add Bars for Frequency Score
fig.add_trace(go.Bar(
    x=segment_scores['RFM_Customer_Segments'],
    y=segment_scores['F'],
    name='Frequency Score',
    marker_color='rgb(94,158,217)'
))

#add Bars for Recency Score
fig.add_trace(go.Bar(
    x=segment_scores['RFM_Customer_Segments'],
    y=segment_scores['M'],
    name='Monetary Score',
    marker_color='rgb(32,102,148)'
))

#Update LAyout
fig.update_layout(
    title='Comparison of RFM Segments Based on Recency, Frequency, and Monetary Scores',
    xaxis_title='RFM Segments',
    yaxis_title='Score',
    barmode='group',
    showlegend=True
)

fig.show()